# 卷积神经网络

## 1、前言

经过前面几节的学习，终于完成了我们的基础部分，下面正式进入到了深度学习部分。

第一个要讲的当然是 cnn 了，也就是卷积神经网络，具体的就不再仔细介绍了，我们直接进入代码部分。

## 2、模型

数据集仍然是使用 MNIST 手写字体，和之前一样做同样的预处理。

### 2.1、导入库和包

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

### 2.2、定义模型

In [2]:
# 定义超参数
batch_size = 128
learning_rate = 1e-2
num_epoches = 5

# 定义 Convolution Network 模型
class Cnn(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Cnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
        )

        self.fc = nn.Sequential(
            nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, n_class)
        )

    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = Cnn(1, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

以上就是网络的模型部分了。和之前相比主要增加了这些不一样的部分。

 - 1、 nn.Sequential()
 这个表示将一个有序的模块写在一起，也就相当于将神经网络的层按顺序放在一起，这样可以方便结构显示。
 - 2、nn.Conv2d()
 这个是卷积层，里面常用的参数有四个，in_channels,out_channels,kernel_size,stride, padding
 in_channels 表示的是输入卷积层的图片的厚度
 out_channels 表示的是要输出的厚度
 kernel_size 表示的是卷积核的大小，可以用一个数字表示长宽相等的卷积核，比如 kernel_size=3 ,也可以用不同的数字表示长宽不同的卷积核，比如 kernel_size=(3,2) 
 stride 表示卷积核滑动的步长
 padding 表示的是图片周围填充 0 的多少，padding=0 表示不填充，padding=1 表示四周都填充 1维
 - 3、nn.ReLU()
 这个表示使用 ReLU() 激活函数，里面有一个参数 inplace，默认设置为 False，表示新创建一个对象对其修改，也可以设置为 True，表示直接对这个对象进行修改。
 - 4、nn.MaxPool2d()
 这个是最大池化层，当然也有平均池化层，里面的参数有 kernel_size，stride，padding
 kernel_size 表示池化的窗口大小，和卷积里面的 kernel_size 是一样的
 stride 也和卷积层里面一样，需要自己设置滑动步长
 padding也和卷积层里面的参数是一样的，默认是 0
 模型需要传入的参数是输入的图片维数以及输出的种类数
 
### 2.3、训练

训练过程是一样的，只是输入的图片不再需要展开

这是训练 20 个 epoch 的结果，当然你也可以增加训练次数，修改里面的参数达到更好的效果，可以参考一下 LeNet 的网络结构，自己重新写一下

![](img/pt_4_1.jpg)

大体上简单的卷积神经网络就是这么构建的，当然现在也有很多复杂的网络，比如 vgg，inceptionv1-v4，resnet 以及修正的 inception-resnet，这些网络都是深层的卷积网络，有兴趣的同学可以去看看 pytorch 的官方代码实现，或者去 github 上搜索相应的网络。